# FMRIPREP Report 

In [1]:
# import pacakges 

import os, glob, shutil
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
import subprocess

from IPython.display import SVG, display
from datetime import date

pd.set_option('display.max_rows', 159)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', 159)
#pd.set_option('display.max_colwidth', -1)
from matplotlib import rcParams
rcParams['axes.titlepad'] = 15

%matplotlib inline 

## Set Variables

In [2]:
# Variable load and setup
date = date.today()
study_folder_path="/projects/niblab/experiments/bbx"
data_folder_path= os.path.join(study_folder_path, "data")
report_folder_path=os.path.join(data_folder_path, "quality_analysis/reports")
fmriprep_path='/projects/niblab/experiments/bbx/data/fmriprep'
fmriprep_folders=glob.glob(os.path.join(fmriprep_path, "sub-*/ses-*"))
fmriprep_ses1=[x.split("/")[-2] for x in fmriprep_folders if 'ses-1' in x]
fmriprep_ses2=[x.split("/")[-2] for x in fmriprep_folders if 'ses-2' in x]


# Load report file
# get subject count
data_report_tsv=os.path.join(data_folder_path,
                                  "quality_analysis/reports/bbx_ses-1_preprocessing_report.tsv")

data_report=pd.read_csv(data_report_tsv, sep="\t",
                           index_col="subject")



runs=['run-1','run-2','run-3', 'run-4']
sessions=['ses-1','ses-2']
tasks=['task-training']
subject_list=list(set(fmriprep_ses1+fmriprep_ses2))

subject_list.sort()


In [3]:
subject_list[:4]

['sub-001', 'sub-002', 'sub-003', 'sub-004']

In [4]:
"""
# Methods
"""

def quality_report():
    s1_bids_subject_list=[x.split("/")[-2] for x in 
                   glob.glob(os.path.join(study_folder_path, "data/bids/sub-*/ses-1"))]


    s2_bids_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(study_folder_path, "data/bids/sub-*/ses-2"))]


    s1_fprep_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(study_folder_path, "data/fmriprep/sub-*/ses-1"))]


    s2_fprep_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(study_folder_path, "data/fmriprep/sub-*/ses-2"))]
    s1_fprep_missing = [x for x in s1_bids_subject_list if x not in s1_fprep_subject_list]
    s2_fprep_missing = [x for x in s2_bids_subject_list if x not in s2_fprep_subject_list]

    fprep_missing=list(set(s1_fprep_missing + s2_fprep_missing))
    fprep_missing.sort()

    print("[INFO] %s session 1 fmriprep subjects found."%len(s1_fprep_subject_list))
    print("[INFO] %s session 1 bids subjects are not processed through fmriprep."%len(s1_fprep_missing))
    print("[INFO] subjects: %s "%s1_fprep_missing)
    print("\n[INFO] %s session 2 fmriprep subjects found."%len(s2_fprep_subject_list))
    print("[INFO] %s session 2 bids subjects are not processed through fmriprep."%len(s2_fprep_missing))
    print("[INFO] subjects: %s "%s2_fprep_missing)
    print("\n[INFO] set of missing subjects: %s"%fprep_missing)
    
    
"""
# Build BIDS Report 

"""


def build_fmriprep_report(write_files=False):
    sessions=['ses-1', 'ses-2']
    # loop through sessions
    for session in sessions:
        #print("\n[INFO] %s"%session)


        data_dict={} #initialize data dictionary for session

        # loop through subject set by subject
        for i in range(1,171):
            subject="sub-%s"%f'{i:03}'
            bids_folder=os.path.join(fmriprep_path, subject,
                                    session)

            if subject not in data_dict:
                data_dict[subject] = {}



            # get anat file and save plot
            #_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.g
            anat_img =os.path.join(fmriprep_path, subject, "anat",
                   '%s_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.g'%(subject))

            if os.path.exists(anat_img): 
                data_dict[subject]["anat"]="good"

            # get functional files and check their volume and plot images
            func_files=glob.glob(os.path.join(
                    fmriprep_path, subject, session,
                    "func/*task-training_run-*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" ))

            for func_file in func_files:
                task=func_file.split("/")[-1].split("_")[2]
                vol = subprocess.check_output(["fslnvols", func_file])
                vol=str(vol,'utf-8').strip("\n")


                if "resting" in task:
                    var_name="%s_vol"%task
                    data_dict[subject][var_name]=vol
                elif "rl" in task:
                    run=func_file.split("/")[-1].split("_")[3]
                    var_name="%s_%s_vol"%(task,run)
                    data_dict[subject][var_name]=vol
                elif "training" in task:
                    run=func_file.split("/")[-1].split("_")[3]
                    var_name="%s_%s_vol"%(task,run)
                    data_dict[subject][var_name]=vol


            # plot functionals
            #pool = Pool()
            #pool.map(plot_functionals, func_files)
            #pool.close()

        #excel_file=os.path.join(report_folder_path, "bbx_bids_report.xlsx")
        dataframe=pd.DataFrame(data_dict).T
        display(dataframe)  
        #print(excel_file)
        #dataframe.to_excel(excel_file, sheet_name="%s"%session)
        
        
    

## Submit FMRIPREP batch

In [5]:
def submit_fmriprep_batch(job_file, sub, sess, x,y,z, submit_job=False):
    print('[INFO] batch file: %s'%job_file)
    id_int=sub.split("-")[1].lstrip('0')
    batch_cmd='sbatch --array={}-{}%{} {} 1'.format(x,y,z, job_file)
    print('[INFO] batch command: {}'.format(batch_cmd))
    
    # submit batch job
    if submit_job==True: 
        sp.run(batch_cmd, shell=True)
        print('[INFO] submitted job.')

Set variables for batch command and submit job:  
`sbatch --array={start_id}-{finish_id}%{z} {job_file}`

In [6]:
# Set Parameters 
fmriprep_job_file='/projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job'
start_id=1
finish_id=1
z=1 # how many jobs to run
sess=1
submit_job=False # set to True when you want to run the file 
sub="sub-001"
submit_fmriprep_batch(fmriprep_job_file, sub, sess,start_id, finish_id, z, submit_job)

[INFO] batch file: /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job
[INFO] batch command: sbatch --array=1-1%1 /projects/niblab/experiments/bbx/code/preprocessing/fmriprep/fmriprep.job 1


## Quality Report

In [7]:
quality_report()

[INFO] 149 session 1 fmriprep subjects found.
[INFO] 1 session 1 bids subjects are not processed through fmriprep.
[INFO] subjects: ['sub-074'] 

[INFO] 125 session 2 fmriprep subjects found.
[INFO] 6 session 2 bids subjects are not processed through fmriprep.
[INFO] subjects: ['sub-016', 'sub-035', 'sub-059', 'sub-074', 'sub-094', 'sub-128'] 

[INFO] set of missing subjects: ['sub-016', 'sub-035', 'sub-059', 'sub-074', 'sub-094', 'sub-128']


In [8]:
fmriprep_dict={}


In [9]:
print("\n[INFO] fmriprep qc report")
print("[INFO] %s bids subject folders found for session 1."%len(fmriprep_ses1))
print("[INFO] %s bids subject folders found for session 2."%len(fmriprep_ses2))



[INFO] fmriprep qc report
[INFO] 149 bids subject folders found for session 1.
[INFO] 125 bids subject folders found for session 2.


In [10]:
build_fmriprep_report(write_files=False)

,task-training_run-4_vol,task-training_run-3_vol,task-training_run-2_vol,task-training_run-1_vol
sub-001,233,233,233,233
sub-002,233,233,233,233
sub-003,233,233,233,233
sub-004,233,233,233,233
sub-005,233,233,233,233
...,...,...,...,...
sub-166,233,233,233,233
sub-167,233,233,233,233
sub-168,233,233,233,233
sub-169,233,233,233,233


,task-training_run-4_vol,task-training_run-3_vol,task-training_run-2_vol,task-training_run-1_vol,task-training_run-5_vol
sub-001,233,233,233,233,NaN
sub-002,233,233,233,233,NaN
sub-003,NaN,NaN,NaN,NaN,NaN
sub-004,233,233,233,233,NaN
sub-005,233,233,233,233,NaN
...,...,...,...,...,...
sub-166,NaN,NaN,NaN,NaN,NaN
sub-167,NaN,NaN,NaN,NaN,NaN
sub-168,NaN,NaN,NaN,NaN,NaN
sub-169,NaN,NaN,NaN,NaN,NaN


In [11]:
def display_fmriprep(sub_id=subject_list[0], sess_id=sessions[0]):
    print("[INFO] viewing report for %s"%sub_id)
    fmriprep_path=os.path.join(study_folder_path,'data/fmriprep')
    
    # anat figures
    anat_imgs=glob.glob(os.path.join(fmriprep_path, '%s/figures/*'%sub_id))
    anat_t1w=os.path.join(fmriprep_path, '%s/figures/%s_space-MNI152NLin2009cAsym_T1w.svg'%(sub_id,sub_id))
    try:
        display(SVG(filename=anat_t1w))
        #for svg in anat_imgs:
            #print("[INFO] viewing file %s"%svg)
            #display(SVG(filename=svg))
    except:
        print("[INFO] no anat figures found for %s"%sub_id)

# func figures
    print('\n[INFO] functional reports for %s'%sess_id)
    sess_svgs=glob.glob(os.path.join(fmriprep_path, '%s/%s/figures/*'%(sub_id,sess_id)))
    try:
        for svg in sess_svgs:
            if "flirt" in svg:
            #"brain_mask" in svg or "magnitude_bold" in svg or "rois" in svg or \
            #"flirt" in svg or "sdc" in svg or "fieldmap" in svg:
                print("[INFO] viewing file %s"%svg)
                display(SVG(filename=svg))
    except:
        print("[INFO] no anat figures found for %s"%sub_id)


In [12]:
# run widget that calls help functions above
w_fmriprep = widgets.Select(options=subject_list)
widgets.interactive(display_fmriprep, sub_id=subject_list, sess_id=sessions)

interactive(children=(Dropdown(description='sub_id', options=('sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub…